# Exploration of Random Forest

In [10]:
import os

import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import itertools  

from sklearn.ensemble import RandomForestRegressor

from pprint import pprint

import matplotlib.pyplot as plt
%matplotlib inline 
import time, math

In [11]:
IMC_basedir = os.getcwd().split('code')[0]
DATA_FILE = os.path.join(IMC_basedir, 'data/imputed_dataset_no_censoring_26022018_Amelia1.csv')
DATA_MICE_FILE = os.path.join(IMC_basedir, 'data/imputed_dataset_no_censoring_26022018_MICE')
TRAIN_FILE = os.path.join(IMC_basedir, 'data/amelia_train')
TEST_FILE = os.path.join(IMC_basedir, 'data/amelia_test')
MODEL_DIR = os.path.join(IMC_basedir, 'data/amelia_model')

In [12]:
def binning(col, cut_points, labels=None):
    #Define min and max values:
    minval = col.min()
    maxval = col.max()

    #create list by adding min and max to cut_points
    break_points = [minval] + cut_points + [maxval]

    # if no labels provided, use default labels 0 ... (n-1)
    if not labels:
        labels = range(len(cut_points)+1)

    #Binning using cut function of pandas
    colBin = pd.cut(col,bins=break_points,labels=labels,include_lowest=True)
    return colBin

def plot_report(Y_true, Y_pred):

    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(confusion_matrix(Y_test, Y_pred), 
                          classes=labels, title='Confusion matrix')

    print(classification_report(Y_test, Y_pred, target_names=labels))
    print('Accuracy: {}'.format(accuracy_score(Y_test, Y_pred)))

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def plot_hist(history, save_as=None):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    if save_as:
        plt.savefig(save_as + '_acc.jpg')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    if save_as:
        plt.savefig(save_as + '_loss.jpg')
    plt.show()

In [13]:
df_mice = pd.read_csv(DATA_MICE_FILE)
df_mice.drop("Unnamed: 0", axis = 1, inplace=True)

In [14]:
# Load the dataset
df_amelia = pd.read_csv(DATA_FILE)
df_amelia.drop("Unnamed: 0", axis = 1, inplace=True)
labels = ["1.5year","4years","more"]
cut_points = [500,1500]

#labels = ["3_months","6_months","9_months","12_months","15_months","18_months","2_years","3_years","4_years","5_years","10_years","10_plus_years"]
#cut_points = [90,180,270,360,450,540,720,1095,1460,1825,3650]
df_amelia.loc[:,"life_expectancy_bin"] = binning(df_amelia.life_expectancy, cut_points, labels)

df_amelia['life_expectancy_bin'] = LabelEncoder().fit_transform(df_amelia['life_expectancy_bin'])
#df_amelia.drop("life_expectancy", axis=1, inplace =True)

In [15]:
le_dict = dict() # Initialise an empty dictionary to keep all LabelEncoders
df_categories = df_amelia.copy(deep=True) 
# Loop over attributes by excluding the continuous oness
for column in df_categories.drop(['Age_surgery', 'life_expectancy', 'Tumor_grade','IDH_TERT','IK'], axis=1):  
    le = LabelEncoder().fit(df_categories[column]) # Initialise the LabelEncoder and fit
    df_categories[column] = le.transform(df_categories[column]) # Transform data and save in credit_clean DataFrame
    le_dict[column] = le # Store the LabelEncdoer in dictionary
    
df = df_amelia.copy(deep=True)
non_dummy_cols = ['Tumor_grade','IDH_TERT','life_expectancy','life_expectancy_bin','Gender','IK','Age_surgery']
dummy_cols = list(set(df.columns) - set(non_dummy_cols))

df = pd.get_dummies(df,columns=dummy_cols)

df.Gender.replace(to_replace={'M':1, 'F':0},inplace=True)

In [16]:
def get_train_test_data(data_df, train_size=0.8):
    df_tensorflow = data_df.copy(deep=True)
    X_train, X_test = train_test_split(df_tensorflow, train_size=train_size, 
                                       test_size=1-train_size)

    # remove columns
    X_train.drop('life_expectancy', axis = 1, inplace=True)
    X_test.drop('life_expectancy', axis = 1, inplace=True)

    Y_train = X_train['life_expectancy_bin']
    X_train.drop('life_expectancy_bin', axis = 1, inplace=True)

    Y_test = X_test['life_expectancy_bin']
    X_test.drop('life_expectancy_bin', axis = 1, inplace=True)
    
    return X_train, Y_train, X_test, Y_test

## 1.1 Classification: Benchmark Logistic Regression (Scikit-learn)

In [17]:
X_train, Y_train, X_test, Y_test = get_train_test_data(df, train_size=0.8)

In [20]:
rf = RandomForestRegressor(random_state = 182)

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 182,
 'verbose': 0,
 'warm_start': False}


In [24]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [27]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(random_state = 42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 5, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

In [28]:
# Fit the random search model
rf_random.fit(X_train, Y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_depth=50, max_features=sqrt, bootstrap=True 
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_depth=50, max_features=sqrt, bootstrap=True 
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_depth=50, max_features=sqrt, bootstrap=True 
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_depth=90, max_features=sqrt, bootstrap=False 
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_depth=90, max_features=sqrt, bootstrap=False 
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_depth=90, max_features=sqrt, bootstrap=False 
[CV] n_estimators=600, min_samples_split=2, min_samples_leaf=2, max_depth=60, max_features=auto, bootstrap=False 
[CV] n_estimators=600, min_samples_split=2, min_samples_leaf=2, max_depth=60, max_features=auto, bootstrap=False 
[CV]  n_estimators=200, m

[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    5.4s remaining:    4.7s


[CV]  n_estimators=600, min_samples_split=2, min_samples_leaf=2, max_depth=60, max_features=auto, bootstrap=False, total=   6.2s
[CV]  n_estimators=600, min_samples_split=2, min_samples_leaf=2, max_depth=60, max_features=auto, bootstrap=False, total=   6.2s
[CV]  n_estimators=1400, min_samples_split=5, min_samples_leaf=1, max_depth=30, max_features=sqrt, bootstrap=True, total=   4.3s
[CV]  n_estimators=600, min_samples_split=2, min_samples_leaf=2, max_depth=60, max_features=auto, bootstrap=False, total=   6.1s
[CV]  n_estimators=1000, min_samples_split=10, min_samples_leaf=1, max_depth=80, max_features=auto, bootstrap=False, total=   7.7s
[CV]  n_estimators=1000, min_samples_split=10, min_samples_leaf=1, max_depth=80, max_features=auto, bootstrap=False, total=   7.7s
[CV]  n_estimators=1000, min_samples_split=10, min_samples_leaf=1, max_depth=80, max_features=auto, bootstrap=False, total=   6.4s


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   11.9s finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=5, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_leaf': [1, 2, 4], 'max_features': ['auto', 'sqrt'], 'bootstrap': [True, False], 'min_samples_split': [2, 5, 10]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring='neg_mean_absolute_error',
          verbose=2)

In [29]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 30,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 1400}

In [30]:
rf_random.cv_results_

{'mean_fit_time': array([0.57640791, 1.81294592, 6.09864934, 4.13554637, 7.15006399]),
 'mean_score_time': array([0.02936673, 0.08575805, 0.07761224, 0.21870097, 0.09134579]),
 'mean_test_score': array([-0.50686467, -0.50366319, -0.54021666, -0.50328853, -0.52879843]),
 'mean_train_score': array([-0.39001478, -0.36701985, -0.12369554, -0.29812566, -0.21711964]),
 'param_bootstrap': masked_array(data=[True, False, False, True, False],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[50, 90, 60, 30, 80],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_features': masked_array(data=['sqrt', 'sqrt', 'auto', 'sqrt', 'auto'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_leaf': masked_array(data=[2, 4, 2, 1, 1],
              mask=[False, False,

#### Evaluate

In [31]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [33]:
# Evaluate baseline
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, Y_train)
base_accuracy = evaluate(base_model, X_test, Y_test)

Model Performance
Average Error: 0.4770 degrees.
Accuracy = -inf%.


In [34]:
# Evaluate the Best Random Search Model
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, Y_test)

Model Performance
Average Error: 0.4713 degrees.
Accuracy = -inf%.


In [35]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Improvement of nan%.


/Users/paul.pop/miniconda3/envs/KS3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


## Grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# Create a base model
rf = RandomForestRegressor(random_state = 42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, Y_train);

In [ ]:
grid_search.best_params_

In [ ]:
#### Evaluate

In [ ]:

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, test_features, test_labels)

In [ ]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(13, kernel_initializer="normal", input_dim=54, activation="relu"))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
#test_kfolds(KerasRegressor(build_fn=larger_model, nb_epoch=50, batch_size=5, verbose=0))

In [ ]:
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=54, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
#test_kfolds(KerasRegressor(build_fn=wider_model, nb_epoch=50, batch_size=5, verbose=0))

In [ ]:
def mlp_model():
    model = Sequential()
    
    model.add(Dense(50, input_dim = 54, kernel_initializer='he_normal', activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))    
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, kernel_initializer='he_normal'))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:

# create 5 models to ensemble
model1 = KerasClassifier(build_fn = mlp_model, epochs = 100)
model2 = KerasClassifier(build_fn = mlp_model, epochs = 100)
model3 = KerasClassifier(build_fn = mlp_model, epochs = 100)
model4 = KerasClassifier(build_fn = mlp_model, epochs = 100)
model5 = KerasClassifier(build_fn = mlp_model, epochs = 100)

In [ ]:
ensemble_clf = VotingClassifier(estimators = [('model1', model1), ('model2', model2), ('model3', model3), ('model4', model4), ('model5', model5)], voting = 'soft')

In [ ]:
ensemble_clf.fit(X_train, Y_train)

In [ ]:
X_train, Y_train, X_test, Y_test = get_train_test_data(df)

In [ ]:
model = Sequential()
model.add(Dense(50, kernel_initializer="normal", input_dim=54))
model.add(Activation('relu'))
model.add(Dense(1))
    
model.compile(optimizer = optimizers.SGD(lr = 0.001), loss = 'mean_squared_error', metrics = ['accuracy'])

history = model.fit(np.array(X_train), np.array(Y_train), epochs = 100, verbose = 1)

# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.legend(['training', 'validation'], loc = 'upper left')
# plt.show()

results = model.evaluate(X_test, Y_test)
print('Test accuracy: ', results[1])

In [ ]:
model.summary()

In [ ]:
# Categorical predictions
from keras.utils.np_utils import to_categorical
X = df.copy(deep=True)

Y = np.array(to_categorical(X['life_expectancy_bin']))
# remove columns
X.drop('life_expectancy', axis = 1, inplace=True)
X.drop('life_expectancy_bin', axis = 1, inplace=True)

X = np.array(X)

In [ ]:
seed = 7
np.random.seed(seed)

model = Sequential()
# model.add(Dense(108, input_shape=(54,), kernel_initializer='normal', activation='relu'))
# model.add(BatchNormalization())
# model.add(Dense(32, kernel_initializer='normal', activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(16, kernel_initializer='normal', activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(8, kernel_initializer='normal', activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(64, activation='relu', input_dim=54))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# model.add(Dense(input_dim=54, output_dim=12, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(input_dim=12, output_dim=12, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(output_dim=1, activation='softmax'))
# model.compile(loss='mean_squared_error', optimizer='adadelta', metrics=['accuracy'])

# 3
#tbCallBack = keras.callbacks.TensorBoard(log_dir='/tmp/keras_logs', write_graph=True)

# 4
#model.compile(loss='mean_squared_error', optimizer='adadelta', metrics=['accuracy'])

model.summary()

hist = model.fit(X, Y, epochs=600, batch_size=128,  verbose=1, validation_split=0.4)#, callbacks=[tbCallBack])

In [ ]:
plot_hist(hist)

In [ ]:
plot_hist(hist) #, save_as='deep_5_wide_108')

## 2. Scikit-Neural Network (SKNN)

Different approach for Classification vs. Regression using Neural Network:
- Training examples: Rn x {class_1, ..., class_n} (one-hot encoding) vs Rn x Rm
- Last layer: softmax vs linear / sigmoid
- Loss function: Cross entropy vs MSE / Absolute error

### 2.2. SKNN: Regression (sine function example)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

X, Y = make_regression(n_features=54, n_informative=2,random_state=0, shuffle=False)

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [ ]:
print(regr.predict(X_test))

In [8]:
from sknn.mlp import Regressor, Layer
from sklearn.cross_validation import train_test_split

# Build the dataset
sine_X = np.linspace(0, 2 * 2 * math.pi, 1001)  
sine_y = 5 * np.sin(sine_X)
sine_X_train, sine_X_test, sine_y_train, sine_y_test = train_test_split(sine_X, sine_y, test_size=0.33, random_state=42)
# Transform 1D arrays to 2D arrays
sine_X_train = sine_X_train[:,None]
sine_X_test = sine_X_test[:,None]
sine_y_train = sine_y_train[:,None]
sine_y_test = sine_y_test[:,None]

# Build the NN
nn01 = Regressor(layers=[# input layer is added automatically, based on the number of features
                        # a bias unit is added automatically for every input and hiddens layers
                        Layer("Tanh", units=30),      # 1st hidden layer: Rectifier, Sigmoid or Tanh
                        Layer("Tanh", units=30),      # 2nd hidden layer: Rectifier, Sigmoid or Tanh
                        Layer("Linear", units=1)],    # output layer                           
                learning_rate=0.001,
                weight_decay=0.1,                     # weight_decay = regularization
                regularize='L2',
                learning_momentum=0.66,
                n_iter=50,
                batch_size=1,                         # batch_size=1: each sample is treated on its own
                loss_type='mse',
                verbose=True)

# Fitting the model
nn01.fit(sine_X_train, sine_y_train)

# Making the prediction
sine_y_pred = nn01.predict(sine_X_test) 

# Results
print("Results of SKNN Regression:")
print("Residual sum of squares (MSE): %.2f" % np.mean((sine_y_pred - sine_y_test) ** 2))
print('Variance score: %.2f' % nn01.score(sine_X_test, sine_y_test)) # Explained variance score: 1 is perfect prediction

# Plot outputs
plt.clf() 
plt.scatter(sine_X_train, sine_y_train,  color='black', label='training data', alpha=0.1)
plt.scatter(sine_X_test, sine_y_test,  color='red', label='test data')
plt.scatter(sine_X_test, sine_y_pred, color='blue', label='NN prediction') ## NN
plt.grid(True)
plt.legend()
plt.show()

ImportError: cannot import name 'downsample'

#### Randomized Search for hyperparameter tuning

In [ ]:
# Specify parameters and distributions to sample from
param_dist = {  "learning_rate"     : np.logspace(-4, -2, 3),  # 3 numbers from 1e-4 to 1e-2
                "weight_decay"      : np.logspace(-4, -2, 3),
                "learning_momentum" : [0.33, 0.66, 0.99],
                "n_iter"            : [30, 40, 50]}

# Start the Randomized Search
n_iter_search = 20
random_search = RandomizedSearchCV(nn01, param_distributions=param_dist, n_iter=n_iter_search, 
                                   scoring='mean_squared_error', n_jobs=-1, cv=3, verbose=3)

random_search = random_search.fit(sine_X_train, sine_y_train)
print "Best parameters set found on development set:"
print random_search.best_score_, random_search.best_params_
print
print "Grid scores on development set:"
for params, mean_score, scores in random_search.grid_scores_:
    print "%0.3f (+/-%0.03f) for %r" % (mean_score, scores.std() * 2, params)

In [ ]:
nn01_opt = Regressor(layers=[# input layer is added automatically, based on the number of features
                             # a bias unit is added automatically for every input and hiddens layers
                             Layer("Tanh", units=30),      # 1st hidden layer: Rectifier, Sigmoid or Tanh
                             Layer("Tanh", units=30),      # 2nd hidden layer: Rectifier, Sigmoid or Tanh
                             Layer("Linear", units=1)],    # output layer                           
                learning_rate=random_search.best_params_['learning_rate'],  
                weight_decay=random_search.best_params_['weight_decay'],      
                regularize='L2',
                learning_momentum=random_search.best_params_['learning_momentum'],
                n_iter=random_search.best_params_['n_iter'],
                batch_size=1,    # batch_size=1: each sample is treated on its own
                loss_type='mse',
                verbose=True)

# Fitting the model
nn01_opt.fit(sine_X_train, sine_y_train)

# Plot outputs
plt.clf() 
plt.scatter(sine_X_train, sine_y_train,  color='black', label='training data', alpha=0.1)
plt.scatter(sine_X_test, sine_y_test,  color='red', label='test data')
plt.scatter(sine_X_test, nn01_opt.predict(sine_X_test) , color='blue', label='NN prediction') ## NN
plt.grid(True)
plt.legend()
plt.show()
print "Residual sum of squares (MSE): %.2f" % np.mean((nn01_opt.predict(sine_X_test) - sine_y_test) ** 2)

#### Learning curves

In [ ]:
title = 'Learning Curves (NN with tuned hyperparameters)'    
estimator = nn01_opt # regressor with tuned hyperparameters 
plot_learning_curve(estimator, title, sine_X_train, sine_y_train, 
                    ylim=(-10., 0.), cv=5, n_jobs=-1, scoring='mean_squared_error')
plt.grid(True)
plt.show()

Note: It outputs the negative of the MSE, as it always tries to maximize the score. 

## 3. Pybrain

In [ ]:
# pip install git+https://github.com/pybrain/pybrain.git

### 3.1 PyBrain: Regression (diabetes dataset)

In [ ]:
import pybrain2
from pybrain.structure import SigmoidLayer, LinearLayer, TanhLayer
from pybrain.datasets import SupervisedDataSet
from pybrain.supervised.trainers import BackpropTrainer
import pybrain.tools.shortcuts as pb
import numpy, math

# Build the dataset
xvalues = diabetes_X_train[:,0]   #should be normalized when lots of features
yvalues = diabetes_y_train
ds = SupervisedDataSet(1, 1)
for x, y in zip(xvalues, yvalues):
    ds.addSample((x), (y))
    
# Build the NN
nn1 = pb.buildNetwork(1,  # 1 input node
                   20,    # number of nodes in 1st hidden layer
                   #10,   # number of nodes in 2nd hidden layer
                   1,     # 1 output node
                   bias = False,
                   hiddenclass = SigmoidLayer,
                   outclass = LinearLayer )

# Train the NN
trainer = BackpropTrainer(nn1, ds, learningrate = 0.01, weightdecay=0.01, momentum=0.6) #, verbose = True)
train_mse, validation_mse = trainer.trainUntilConvergence(maxEpochs = 20, continueEpochs=5, validationProportion=0.20)

##### Note on some of the parameters

**validationProportion**: ratio of the dataset that is used for the validation dataset.
If maxEpochs is given, at most that many epochs are trained. Each time validation error hits a minimum, try for continueEpochs # epochs to find a better one.

**Epoch**: one epoch means that every example has been seen once. It is preferable to track epochs rather than iterations since 
the number of iterations depends on the arbitrary setting of batch size. Batchs are used for example in the minibatch method,
for example, for 1000 examples, the NN is trained on examples 1-100, then examples 101-201, etc.

**Momentum**: 0 < m < 1 is a global parameter which must be determined by trial and error. Momentum simply adds a fraction m of the previous weight update to the current one. When the gradient keeps pointing in the same direction, this will increase the size of the steps taken towards the minimum. It is otherefore often necessary to reduce the global learning rate µ when using a lot of momentum (m close to 1). If you combine a high learning rate with a lot of momentum, you will rush past the minimum with huge steps! When the gradient keeps changing direction, momentum will smooth out the variations. Adding a momentum can help to speed up convergence to the minimum by damping oscillations.

In [ ]:
# Make the prediction
y_pred = [ nn1.activate([x]) for x in diabetes_X_test ]  

# Print the weights
def weight_connection(n):
    for mod in n.modules:
        for conn in n.connections[mod]:
            print conn
            for cc in range(len(conn.params)):
                print conn.whichBuffers(cc), conn.params[cc]
#weight_connection(nn1)

# And evaluate:

# Fitting error
plt.clf() 
plt.scatter(diabetes_X_train, diabetes_y_train,  color='black', alpha=0.1)
plt.scatter(diabetes_X_test, diabetes_y_test,  color='red', label = 'target')
plt.scatter(diabetes_X_test, y_pred, color = 'blue', label = 'NN output')  # NN
plt.scatter(diabetes_X_test, diabetes_y_pred_lin, color='green', label='linear model')  # benchmark
plt.legend()
plt.grid(True)
plt.show()

# Learning curves
plt.clf()
plt.plot(range(len(train_mse)), np.sqrt(train_mse), color='blue', label='training error')
plt.plot(range(len(validation_mse)), np.sqrt(validation_mse), color='red', label='validation error')
plt.title('Learning curves: loss(=RMSE) as a function of Epochs')
plt.legend()

### 3.2 PyBrain: Regression (diabetes dataset)

Similar as above (section 3.1), except here we are creating the feeforward neural network from scratch.

In [ ]:
from pybrain.structure import FeedForwardNetwork, LinearLayer, SigmoidLayer, TanhLayer, BiasUnit, FullConnection
from pybrain.supervised.trainers import BackpropTrainer

nn2 = FeedForwardNetwork()

bias = False


## Constructing the input, hidden and output layers:

# In order to use them, we have to add them to the network:
nn2.addInputModule( LinearLayer(1, name = 'in') )      # inLayer
nn2.addModule( SigmoidLayer(5, name = 'hidden0') )     # hiddenLayer
nn2.addOutputModule( LinearLayer(1, name = 'out') )    # outLayer
if bias:
    nn2.addInputModule( BiasUnit(name = 'inbias') )        # bias for input layer
    nn2.addModule( BiasUnit(name = 'bias0') )              # bias for hidden layer

# As with modules, we have to explicitly add them to the network:
theta1 = FullConnection(nn2['in'], nn2['hidden0'])
theta2 = FullConnection(nn2['hidden0'], nn2['out'])
nn2.addConnection( theta1 )    # in_to_hidden connections
nn2.addConnection( theta2 )    # hidden_to_out connections
if bias:
    nn2.addConnection(FullConnection(nn2['inbias'], nn2['hidden0']))
    nn2.addConnection(FullConnection(nn2['bias0'], nn2['out']))

nn2.sortModules()     # making the MLP usable


# Build the dataset ***with bias units***
xvalues = diabetes_X_train[:,0]   # Should be normalized when lots of features
yvalues = diabetes_y_train
ds2 = SupervisedDataSet(1, 1)     # Dataset for Supervised Regression Training
# No need to add here the bias term to the feature matrix: it will be added in the training method
for x, y in zip(xvalues, yvalues):
    ds2.addSample((x), (y))       # ds.addSample((x1, ...., xn), (y)) # for each training example

# shows the nn2 weights
weight_connection(nn2)
    
# Train the NN
trainer = BackpropTrainer(nn2, ds2, learningrate = 0.01, weightdecay=0.01, momentum=0.0) #, verbose = True)
train_mse, validation_mse = trainer.trainUntilConvergence(maxEpochs = 20, continueEpochs=5, validationProportion=0.20)

In [ ]:
# Make the predictions
y_pred = [ nn2.activate([x]) for x in diabetes_X_test ]  

# Evaluate: 

# Plot outputs
plt.clf() 
plt.scatter(diabetes_X_train, diabetes_y_train,  color='black', alpha=0.1)
plt.scatter(diabetes_X_test, diabetes_y_test,  color='red', label = 'target')
plt.scatter(diabetes_X_test, y_pred, color = 'blue', label = 'NN output')
plt.scatter(diabetes_X_test, diabetes_y_pred_lin, color='green', label='linear model')  # benchmark
plt.legend()
plt.grid(True)
plt.show()

# Learning curves
plt.clf()
plt.plot(range(len(train_mse)), np.sqrt(train_mse), color='blue', label='training error')
plt.plot(range(len(validation_mse)), np.sqrt(validation_mse), color='red', label='validation error')
plt.title('Learning curves: loss(=RMSE) as a function of Epochs')
plt.legend()

### 3.2 PyBrain: Regression (sine function example)

In [ ]:
from pybrain.structure import SigmoidLayer, LinearLayer, TanhLayer
from pybrain.tools.shortcuts import buildNetwork
from pybrain.datasets import SupervisedDataSet
from pybrain.supervised.trainers import BackpropTrainer

# Build the dataset
xvalues = numpy.linspace(0, 2 * 2 * math.pi, 1001)  # should be normalized when lots of features
yvalues = 5 * numpy.sin(xvalues)
ds = SupervisedDataSet(1, 1)
for x, y in zip(xvalues, yvalues):
    ds.addSample((x,), (y,))

# Build the network
net = buildNetwork(1,  # 1 input node
                   30, # number of nodes in 1st hiddenlayer
                   30, # number of nodes in 1st hiddenlayer
                   1,  # 1 output node
                   bias = True,
                   hiddenclass = TanhLayer,  # better fit than sigmoid
                   outclass = LinearLayer)    # for regression

# Train the NN
trainer = BackpropTrainer(net, ds, learningrate = 0.0005, weightdecay=0.001, momentum=0.5) #, verbose = True)   
train_mse, validation_mse = trainer.trainUntilConvergence(maxEpochs = 50)

# Evaluate:

# Fitting error
plt.scatter(xvalues, [ net.activate([x]) for x in xvalues ], linewidth = 2,
           color = 'blue', label = 'NN output')
plt.plot(xvalues, yvalues, linewidth = 2, color = 'red', label = 'target')
plt.grid(True)
plt.legend()
plt.show()
#Learning curves
plt.clf()
plt.plot(range(len(train_mse)), np.sqrt(train_mse), color='blue', label='training error')
plt.plot(range(len(validation_mse)), np.sqrt(validation_mse), color='red', label='validation error')
plt.title('Learning curves: loss(=RMSE) as a function of Epochs')
plt.legend()

##### Note on some of the parameters

**weight decay** is the L2 regularization. 0 is no weight decay at all.

**learning rate** gives the ratio of which parameters are changed into the direction of the gradient. The learning rate decreases by lrdecay, which is used to to multiply the learning rate after each training step. The parameters are also adjusted with respect to momentum, which is the ratio by which the gradient of the last timestep is used.  

If **batchlearning** is set, the parameters are updated only at the end of each epoch. Default is False.  

## 4. Other packages

Initial analyses indicate that TensorFlow should bring also performance improvements compared to Theano, although no comprehensive benchmarks have yet been published.  
As the other packages are out already for a while, they have large, active communities and often additional supporting software (examples are the very useful wrappers around Theano like Lasagne, Keras and Blocks that provider higher level abstractions to its engine).